In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Practise').getOrCreate()

In [9]:
df_spark = spark.read.option('header','true').csv('test1.csv', inferSchema = True)

In [10]:
df_spark.show()

+---------+---+----------+------+
|     Name|age|Experience|Salary|
+---------+---+----------+------+
|    Krish| 31|        10| 30000|
|Sudhanshu| 30|         8| 25000|
|    Sunny| 29|         4| 20000|
|     Paul| 24|         3| 20000|
|   Harsha| 21|         1| 15000|
|  Shubham| 23|         2| 18000|
+---------+---+----------+------+



In [11]:
df_spark.printSchema()

root
 |-- Name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- Experience: integer (nullable = true)
 |-- Salary: integer (nullable = true)



In [12]:
df_spark.columns

['Name', 'age', 'Experience', 'Salary']

In [13]:
# [Age,Experience] --> new feature -->independent feature

from pyspark.ml.feature import VectorAssembler
featureassembler=VectorAssembler(inputCols=["age","Experience"],outputCol="Independent Features")

In [14]:

output=featureassembler.transform(df_spark)
#

In [15]:

output.show()

+---------+---+----------+------+--------------------+
|     Name|age|Experience|Salary|Independent Features|
+---------+---+----------+------+--------------------+
|    Krish| 31|        10| 30000|         [31.0,10.0]|
|Sudhanshu| 30|         8| 25000|          [30.0,8.0]|
|    Sunny| 29|         4| 20000|          [29.0,4.0]|
|     Paul| 24|         3| 20000|          [24.0,3.0]|
|   Harsha| 21|         1| 15000|          [21.0,1.0]|
|  Shubham| 23|         2| 18000|          [23.0,2.0]|
+---------+---+----------+------+--------------------+



In [16]:
finalized_data=output.select("Independent Features","Salary")

In [17]:

finalized_data.show()

+--------------------+------+
|Independent Features|Salary|
+--------------------+------+
|         [31.0,10.0]| 30000|
|          [30.0,8.0]| 25000|
|          [29.0,4.0]| 20000|
|          [24.0,3.0]| 20000|
|          [21.0,1.0]| 15000|
|          [23.0,2.0]| 18000|
+--------------------+------+



In [18]:
from pyspark.ml.regression import LinearRegression
##train test split
train_data,test_data=finalized_data.randomSplit([0.70,0.30])
# model building
regressor=LinearRegression(featuresCol='Independent Features', labelCol='Salary')
regressor=regressor.fit(train_data)

In [19]:
# Coefficients
regressor.coefficients

DenseVector([-79.3651, 1693.1217])

In [20]:
# Intercepts
regressor.intercept

15714.285714285847

In [21]:
# Prediction
pred_results=regressor.evaluate(test_data)

In [22]:
pred_results.predictions.show()

+--------------------+------+------------------+
|Independent Features|Salary|        prediction|
+--------------------+------+------------------+
|          [23.0,2.0]| 18000|17275.132275132277|
|          [30.0,8.0]| 25000| 26878.30687830688|
+--------------------+------+------------------+



In [23]:

pred_results.meanAbsoluteError,pred_results.meanSquaredError

(1301.5873015873021, 2026734.9738249248)

In [24]:
pred_results.explainedVariance

23387846.084935904

In [25]:
pred_results.r2

0.8345522470347